In [ ]:
import zipfile
import urllib.request
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import keras
from keras.preprocessing.image import ImageDataGenerator

training_url = "https://storage.googleapis.com/download.tensorflow.org/data/rps.zip"
training_file_name = "rock-paper-scissors.zip"
training_dir = "rock-paper-scissors/training/"
urllib.request.urlretrieve(training_url, training_file_name)
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [ ]:
training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = training_datagen.flow_from_directory(training_dir, target_size=(150, 150), class_mode = 'categorical')

Found 2520 images belonging to 1 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image:
    # 150 x 150 with 3 bytes color
    # This is the fist convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # 2nd convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 3rd convlution
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 4th convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
validation_url = "https://storage.googleapis.com/download.tensorflow.org/data/rps-test-set.zip"
validation_file_name = "validation-rock-paper-scissors.zip"
validation_dir = "rock-paper-scissors/validation/"
urllib.request.urlretrieve(validation_url, validation_file_name)
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [ ]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(150, 150), class_mode = 'categorical')

Found 372 images belonging to 1 classes.


In [ ]:
history = model.fit(train_generator, epochs=25, validation_data = validation_generator, verbose = 1)

Epoch 1/25
79/79 [==============================] - 239s 3s/step - loss: 63127440.0000 - accuracy: 0.3278 - val_loss: 380333152.0000 - val_accuracy: 0.0000e+00
Epoch 2/25
79/79 [==============================] - 254s 3s/step - loss: 1569637376.0000 - accuracy: 0.3302 - val_loss: 6052837888.0000 - val_accuracy: 1.0000
Epoch 3/25
79/79 [==============================] - 248s 3s/step - loss: 10133360640.0000 - accuracy: 0.3397 - val_loss: 18784874496.0000 - val_accuracy: 0.0000e+00
Epoch 4/25
79/79 [==============================] - 252s 3s/step - loss: 37448097792.0000 - accuracy: 0.3270 - val_loss: 78192771072.0000 - val_accuracy: 1.0000
Epoch 5/25
79/79 [==============================] - 253s 3s/step - loss: 104423718912.0000 - accuracy: 0.3397 - val_loss: 163050160128.0000 - val_accuracy: 0.0000e+00
Epoch 6/25
79/79 [==============================] - 236s 3s/step - loss: 241333567488.0000 - accuracy: 0.3302 - val_loss: 254999134208.0000 - val_accuracy: 0.0000e+00
Epoch 7/25
79/79 [===

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  #predicting images
  path = fn
  img = tf.keras.utils.load_img(path, target_size=(150, 150))
  x = tf.keras.utils.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)


Saving paper.jpeg to paper (2).jpeg
Saving rock.jpg to rock (2).jpg
Saving scissors.jpg to scissors (2).jpg
1/1 [==============================] - 0s 169ms/step
paper.jpeg
[[0. 0. 1.]]
1/1 [==============================] - 0s 40ms/step
rock.jpg
[[0. 0. 1.]]
1/1 [==============================] - 0s 66ms/step
scissors.jpg
[[0. 0. 1.]]
